In [1]:
%matplotlib inline

import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import randint, uniform
from sklearn import linear_model, preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import (RandomForestRegressor)
from sklearn.linear_model import Lasso, LogisticRegression, Ridge
from sklearn.metrics import (classification_report, mean_absolute_error, r2_score)
from sklearn.model_selection import (GridSearchCV, RandomizedSearchCV,
                                     train_test_split)
from sklearn.preprocessing import (LabelEncoder, MinMaxScaler, OneHotEncoder,
                                   PolynomialFeatures, RobustScaler,
                                   StandardScaler)
from sklearn.svm import SVC

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

In [2]:
# import data
dataset_og = pd.read_csv('Data\Measurements-Transformed')
# kopie maken indien we iets van de originele data nodig hebben
dataset = dataset_og.copy()
dataset.head(5)

,ID,Sex,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
0,371435.0,0.0,21118.0,0.0,-1.75,0.50,55.0,-2.25,1.0,55.0,-1.75,1.00,110.0,-1.25,0.5,110.0
1,371435.0,0.0,20245.0,0.0,-1.75,0.50,65.0,0.00,0.0,0.0,-1.25,0.50,110.0,0.00,0.0,0.0
2,371435.0,0.0,18099.0,0.0,-1.50,0.50,65.0,0.00,0.0,0.0,-1.00,0.50,110.0,0.00,0.0,0.0
3,402916.0,1.0,13825.0,0.0,-3.50,1.50,180.0,-3.50,1.5,180.0,-3.00,1.50,180.0,-3.00,1.5,180.0
4,402916.0,1.0,9653.0,0.0,-2.00,0.75,175.0,0.00,0.0,0.0,-2.00,0.75,180.0,0.00,0.0,0.0


In [3]:
#drop rijen waar < n meting van zijn en houd van de overige de top n meest recente waardes
n = 2
dataset = dataset.groupby('ID').filter(lambda x: len(x) > (n-1))
dataset = dataset.groupby('ID').head(n)

In [4]:
dataset.head(6)

,ID,Sex,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
0,371435.0,0.0,21118.0,0.0,-1.75,0.50,55.0,-2.25,1.0,55.0,-1.75,1.00,110.0,-1.25,0.50,110.0
1,371435.0,0.0,20245.0,0.0,-1.75,0.50,65.0,0.00,0.0,0.0,-1.25,0.50,110.0,0.00,0.00,0.0
3,402916.0,1.0,13825.0,0.0,-3.50,1.50,180.0,-3.50,1.5,180.0,-3.00,1.50,180.0,-3.00,1.50,180.0
4,402916.0,1.0,9653.0,0.0,-2.00,0.75,175.0,0.00,0.0,0.0,-2.00,0.75,180.0,0.00,0.00,0.0
7,662712.0,0.0,25627.0,3.0,2.25,0.75,90.0,5.25,1.0,90.0,2.75,1.00,95.0,5.75,0.75,95.0
8,662712.0,0.0,22321.0,2.5,2.75,0.75,90.0,5.25,2.0,90.0,2.75,2.00,95.0,5.25,0.75,95.0


In [5]:
#weglaten van de minst gecorreleerde features
# dataset.drop(['Sex', 'Add', 'Axis-Close-R', 'Axis-Close-L'],axis=1, inplace=True)

# #2 rijen naast elkaar zetten

# dataset = dataset.merge(dataset ,on=['ID'], suffixes=['_x', ''])
# dataset = dataset.sort_values(by=['ID', 'Measurement_Age_x'])
# dataset = dataset.drop_duplicates(subset=['ID'], keep='first')
# dataset.head()

In [6]:
#2 rijen naast elkaar zetten

dataset = dataset.merge(dataset ,on=['ID', 'Sex'], suffixes=['_x', ''])
dataset = dataset.sort_values(by=['ID', 'Measurement_Age_x'])
dataset = dataset.drop_duplicates(subset=['ID', 'Sex'], keep='first')
dataset.head()

,ID,Sex,Measurement_Age_x,Add_x,Sph-Far-R_x,Cyl-Far-R_x,Axis-Far-R_x,Sph-Close-R_x,Cyl-Close-R_x,Axis-Close-R_x,Sph-Far-L_x,Cyl-Far-L_x,Axis-Far-L_x,Sph-Close-L_x,Cyl-Close-L_x,Axis-Close-L_x,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
19490,100063.0,1.0,11602.0,0.0,-2.75,1.0,105.0,0.00,0.0,0.0,-3.50,1.75,90.0,0.00,0.0,0.0,13181.0,0.0,-2.75,1.0,105.0,-2.75,1.0,105.0,-3.50,1.75,90.0,-3.50,1.75,90.0
28452,100167.0,0.0,15213.0,0.0,0.75,0.0,0.0,0.75,0.0,0.0,0.75,0.00,0.0,0.75,0.0,0.0,15213.0,0.0,0.75,0.0,0.0,0.75,0.0,0.0,0.75,0.00,0.0,0.75,0.00,0.0
102,100238.0,1.0,20731.0,0.0,0.00,0.0,0.0,1.50,0.5,75.0,0.00,0.00,0.0,1.75,0.5,105.0,23791.0,0.0,2.00,0.5,75.0,2.00,0.5,75.0,2.25,0.50,105.0,2.25,0.50,105.0
23794,100262.0,0.0,17107.0,0.0,0.00,0.0,0.0,1.25,0.0,0.0,0.00,0.00,0.0,1.75,0.0,0.0,19170.0,0.0,1.50,0.0,0.0,1.50,0.0,0.0,2.00,0.00,0.0,2.00,0.00,0.0
38518,100315.0,0.0,1757.0,0.0,2.00,0.0,0.0,0.00,0.0,0.0,3.00,0.00,0.0,0.00,0.0,0.0,2075.0,0.0,2.50,0.0,0.0,2.50,0.0,0.0,3.25,0.00,0.0,3.25,0.00,0.0


In [7]:
#Drop kolom ID
dataset.drop(['ID'],axis=1, inplace=True)


### Linear regression

In [8]:
# Splitsen in features en targets

y = dataset['Sph-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


## Standard scaler
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Linear regression

lreg_sph_far_r = linear_model.LinearRegression()
lreg_sph_far_r.fit(X_train,y_train)
print(lreg_sph_far_r.coef_)
r2 = lreg_sph_far_r.score(X_test,y_test)
print('r2 score = ', r2)

# Modeloptimalisatie en Hyperparameter tuning

# Aanmaken van de hogere orde features
graad = 2

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lreg_poly_sph_far_r = Ridge(alpha=0.5,tol=0.0001,fit_intercept=True)
lreg_poly_sph_far_r.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lreg_poly_sph_far_r.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lreg_poly_sph_far_r.score(X_train_poly,y_train))


# # met L1 regularisatie via Lasso regression
# lreg_poly_sph_far_r = Lasso(alpha=0.001,tol=0.0001,fit_intercept=True)
# lreg_poly_sph_far_r.fit(X_train_poly,y_train)      
  
      
# print('R2 score op test set via L1: ',lreg_poly_sph_far_r.score(X_test_poly,y_test))
  
# # R2 -score via L1 op de trainingset
# print('R2 score op training set via L1: ',lreg_poly_sph_far_r.score(X_train_poly,y_train))    

# # Variëren van de alpha en grafiek

# train_r2 = []
# test_r2 = []


# alphas = np.logspace(-2, 6, 1000)

# for alpha in alphas:
#     lregmodel_poly = Ridge(alpha=alpha,tol=0.0001,fit_intercept=True)
#     lregmodel_poly.fit(X_train_poly,y_train)
#     test_r2.append(lregmodel_poly.score(X_test_poly,y_test))  
#     train_r2.append(lregmodel_poly.score(X_train_poly,y_train))


# # Plot r2
# f, ax = plt.subplots(figsize=(10, 8))
# plt.subplot(2, 1, 1)
# plt.semilogx(alphas, train_r2, label='Train')
# plt.semilogx(alphas, test_r2, label='Test')
# plt.legend(loc='lower left')
# plt.ylim([0, 1.2])
# plt.xlabel('Regularization parameter')
# plt.ylabel('R² Performance')



# Via KernelRidge met een polynomial kernel

# from sklearn.kernel_ridge import KernelRidge

# lreg_sph_far_r = KernelRidge(alpha=0.001, degree=3, gamma=None, kernel='polynomial')
# lreg_sph_far_r.fit(X_train,y_train)


# y_predicted = lreg_sph_far_r.predict(X_train)
# r2 = r2_score(y_train,y_predicted)

# print('training set: ',r2)

# y_predicted = lreg_sph_far_r.predict(X_test)
# r2 = r2_score(y_test,y_predicted)

# print('test set: ',r2)

[ 0.01738009 -0.23764272 -0.1457671   2.37895165  0.0675212  -0.10794023
  0.19662774 -0.03005575  0.07955192  0.15863399  0.02505015 -0.09744577
 -0.01839876 -0.04666971  0.07753983  0.59993471]
r2 score =  0.7600462313709084
dimensie van X_train_poly:  (8758, 153)
dimensie van X_test_poly:  (2190, 153)
R2 score op test set via L2:  0.7807206266472486
R2 score op training set via L2:  0.8093472680132459


In [9]:
# Splitsen in features en targets

y = dataset['Cyl-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


## Standard scaler
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Linear regression

lreg_cyl_far_r = linear_model.LinearRegression()
lreg_cyl_far_r.fit(X_train,y_train)
print(lreg_cyl_far_r.coef_)
r2 = lreg_cyl_far_r.score(X_test,y_test)
print('r2 score = ', r2)

# Modeloptimalisatie en Hyperparameter tuning

# Aanmaken van de hogere orde features
graad = 2

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lreg_poly_cyl_far_r = Ridge(alpha=0.5,tol=0.0001,fit_intercept=True)
lreg_poly_cyl_far_r.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lreg_poly_cyl_far_r.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lreg_poly_cyl_far_r.score(X_train_poly,y_train))

[ 0.01366272 -0.23140685 -0.03939548 -0.02133592  0.36337311 -0.04836248
  0.01169927  0.08211321  0.04009263  0.00215098  0.0554181  -0.00362885
  0.01756516 -0.02948853  0.0079643   0.25756035]
r2 score =  0.5887542128275896
dimensie van X_train_poly:  (8758, 153)
dimensie van X_test_poly:  (2190, 153)
R2 score op test set via L2:  0.6234236176091483
R2 score op training set via L2:  0.6531044846157135


In [52]:
# Splitsen in features en targets

y = dataset['Sph-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


## Standard scaler
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Linear regression

lreg_sph_far_l = linear_model.LinearRegression()
lreg_sph_far_l.fit(X_train,y_train)
print(lreg_sph_far_l.coef_)
r2 = lreg_sph_far_l.score(X_test,y_test)
print('r2 score = ', r2)

# Modeloptimalisatie en Hyperparameter tuning

# Aanmaken van de hogere orde features
graad = 2

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lreg_poly_sph_far_l = Ridge(alpha=0.5,tol=0.0001,fit_intercept=True)
lreg_poly_sph_far_l.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lreg_poly_sph_far_l.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lreg_poly_sph_far_l.score(X_train_poly,y_train))

[ 8.47433432e-03 -2.36962906e-01 -1.48130919e-01  2.71154453e-01
 -2.00667392e-03 -1.36980238e-01 -1.67440792e-01 -1.29989213e-02
  1.23461418e-01  2.18632509e+00  6.90237438e-02 -5.47688624e-02
  2.93841414e-01 -4.93545766e-02  3.05146092e-02  5.96750276e-01]
r2 score =  0.7558198646846204
dimensie van X_train_poly:  (8758, 153)
dimensie van X_test_poly:  (2190, 153)
R2 score op test set via L2:  0.7823831316681447
R2 score op training set via L2:  0.800824017672328


In [11]:
# Splitsen in features en targets

y = dataset['Cyl-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Standard scaler
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Linear regression

lreg_cyl_far_l = linear_model.LinearRegression()
lreg_cyl_far_l.fit(X_train,y_train)
print(lreg_cyl_far_l.coef_)
r2 = lreg_cyl_far_l.score(X_test,y_test)
print('r2 score = ', r2)

# Modeloptimalisatie en Hyperparameter tuning

# Aanmaken van de hogere orde features
graad = 2

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lreg_poly_cyl_far_l = Ridge(alpha=0.5,tol=0.0001,fit_intercept=True)
lreg_poly_cyl_far_l.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lreg_poly_cyl_far_l.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lreg_poly_cyl_far_l.score(X_train_poly,y_train))

[ 0.00915932 -0.2548325  -0.05394657 -0.00974897  0.04384759  0.0044898
  0.00376241 -0.02656881  0.00244542 -0.0130741   0.37638563 -0.04691323
  0.03587714  0.09072509  0.0371198   0.29109223]
r2 score =  0.620089756400874
dimensie van X_train_poly:  (8758, 153)
dimensie van X_test_poly:  (2190, 153)
R2 score op test set via L2:  0.6456010227715455
R2 score op training set via L2:  0.6719776299919638


### Random forest regressor

In [33]:
# Splitsen in features en targets

y = dataset['Sph-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#random forest regressor
number_of_trees = 200
max_number_of_depth = 20
max_number_of_features = 6

RFR_model_sph_far_r = RandomForestRegressor(n_estimators=number_of_trees, max_features=max_number_of_features, max_depth=max_number_of_depth)
RFR_model_sph_far_r.fit(X_train,y_train)

RFR_model_sph_far_r.score(X_test,y_test)

0.7988709544565529

In [43]:
# Splitsen in features en targets

y = dataset['Cyl-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


#random forest regressor
number_of_trees = 200
max_number_of_depth = 10
max_number_of_features = 6

RFR_model_cyl_far_r = RandomForestRegressor(n_estimators=number_of_trees, max_features=max_number_of_features, max_depth=max_number_of_depth)
RFR_model_cyl_far_r.fit(X_train,y_train)

RFR_model_cyl_far_r.score(X_test,y_test)

0.6155663605579961

In [35]:
# Splitsen in features en targets

y = dataset['Sph-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


#random forest regressor
number_of_trees = 200
max_number_of_depth = 20
max_number_of_features = 6

RFR_model_sph_far_l = RandomForestRegressor(n_estimators=number_of_trees, max_features=max_number_of_features, max_depth=max_number_of_depth)
RFR_model_sph_far_l.fit(X_train,y_train)

RFR_model_sph_far_l.score(X_test,y_test)

0.7992083707913193

In [49]:
# Splitsen in features en targets

y = dataset['Cyl-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


#random forest regressor
number_of_trees = 200
max_number_of_depth = 10
max_number_of_features = 6

RFR_model_cyl_far_l = RandomForestRegressor(n_estimators=number_of_trees, max_features=max_number_of_features, max_depth=max_number_of_depth)
RFR_model_cyl_far_l.fit(X_train,y_train)

RFR_model_cyl_far_l.score(X_test,y_test)

0.6503587587706554

In [16]:
dataset.head()

,Sex,Measurement_Age_x,Add_x,Sph-Far-R_x,Cyl-Far-R_x,Axis-Far-R_x,Sph-Close-R_x,Cyl-Close-R_x,Axis-Close-R_x,Sph-Far-L_x,Cyl-Far-L_x,Axis-Far-L_x,Sph-Close-L_x,Cyl-Close-L_x,Axis-Close-L_x,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
19490,1.0,11602.0,0.0,-2.75,1.0,105.0,0.00,0.0,0.0,-3.50,1.75,90.0,0.00,0.0,0.0,13181.0,0.0,-2.75,1.0,105.0,-2.75,1.0,105.0,-3.50,1.75,90.0,-3.50,1.75,90.0
28452,0.0,15213.0,0.0,0.75,0.0,0.0,0.75,0.0,0.0,0.75,0.00,0.0,0.75,0.0,0.0,15213.0,0.0,0.75,0.0,0.0,0.75,0.0,0.0,0.75,0.00,0.0,0.75,0.00,0.0
102,1.0,20731.0,0.0,0.00,0.0,0.0,1.50,0.5,75.0,0.00,0.00,0.0,1.75,0.5,105.0,23791.0,0.0,2.00,0.5,75.0,2.00,0.5,75.0,2.25,0.50,105.0,2.25,0.50,105.0
23794,0.0,17107.0,0.0,0.00,0.0,0.0,1.25,0.0,0.0,0.00,0.00,0.0,1.75,0.0,0.0,19170.0,0.0,1.50,0.0,0.0,1.50,0.0,0.0,2.00,0.00,0.0,2.00,0.00,0.0
38518,0.0,1757.0,0.0,2.00,0.0,0.0,0.00,0.0,0.0,3.00,0.00,0.0,0.00,0.0,0.0,2075.0,0.0,2.50,0.0,0.0,2.50,0.0,0.0,3.25,0.00,0.0,3.25,0.00,0.0


In [17]:
values = np.array([[0, 1757.0, 0.0, 2.00, 0.0, 0.0, 0.00, 0.0, 0.0, 3.00, 0.00, 0.0, 0.00, 0.0, 0.0, 2075.0]])
#Sph-Far-R = 2.50
#Cyl-Far-R = 0
#Sph-Far-L = 3.25
#Cyl-Far-L = 0

# values = values.reshape(-1, 1)
# scaler1.fit(values)
# values = scaler1.transform(values)

models = [lreg_sph_far_r, lreg_cyl_far_r, lreg_sph_far_l, lreg_cyl_far_l,
          RFR_model_sph_far_r, RFR_model_cyl_far_r, RFR_model_sph_far_l, RFR_model_cyl_far_l]

for n in models:
    x = n.predict(values)
#     a = np.array([0, 0, 0, x, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
#     x = scaler1.inverse_transform(x)
    print(x)


[832.52829431]
[128.29221006]
[828.94126251]
[156.69290813]
[2.50875]
[0.]
[3.525]
[0.015]


In [18]:
X_train.head()

,Sex,Measurement_Age_x,Add_x,Sph-Far-R_x,Cyl-Far-R_x,Axis-Far-R_x,Sph-Close-R_x,Cyl-Close-R_x,Axis-Close-R_x,Sph-Far-L_x,Cyl-Far-L_x,Axis-Far-L_x,Sph-Close-L_x,Cyl-Close-L_x,Axis-Close-L_x,Measurement_Age
28752,1.0,23100.0,0.00,0.00,0.50,180.0,0.00,0.50,180.0,0.00,0.25,135.0,0.00,0.25,135.0,23100.0
22542,0.0,28244.0,2.75,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,2.75,0.00,0.0,28580.0
23564,0.0,15853.0,0.00,-3.50,1.75,90.0,-3.50,1.75,90.0,-2.50,0.75,80.0,-2.50,0.75,80.0,15853.0
37586,0.0,19761.0,2.00,1.25,0.00,0.0,3.25,0.00,0.0,1.25,0.00,0.0,3.25,0.00,0.0,19810.0
20486,0.0,8605.0,0.00,-1.25,0.00,0.0,0.00,0.00,0.0,-1.00,0.00,0.0,0.00,0.00,0.0,9903.0
